# A.07 布尔数组和掩码

本节讨论布尔掩码的使用

### 7.1 示例：雨天计数

现有Seattle市，2014年，每天降雨量的系列数据

使用Pandas导入数据，详见[Pandas 介绍](http://pandas.pydata.org/)。

In [ ]:
!type data\Seattle2014.csv

In [ ]:
import numpy as np
import pandas as pd

# use pandas to extract rainfall inches as a NumPy array
rainfall = pd.read_csv('data/Seattle2014.csv')['PRCP'].values
inches = rainfall / 254.0  # 1/10mm -> inches
inches.shape

#### 观察数据及其柱图

- 数组含有365组值，从2014年元旦到岁末的日降雨数据
- 观察其柱状分布图，该图的绘制由Matplotlib完成

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # set plot styles

In [ ]:
plt.hist(inches, 40);

柱状图给出了数据的分布特征，尽管该城以多雨著称，但图中显示2014年绝大多数天的降雨量都接近零。

仅看出这样还不够，要考虑许多问题，如
- 当年有多少雨天？
- 这些天的平均降雨量是多少？
- 有多少天的降雨量超过半英寸？

#### 挖掘数据

- 立即想到的办法：循环数据，采用计数器统计
- 印象：效率太低（写代码，计算）

#### 提高效率

- 采用NumPy的通用函数
- 另一类通用函数进行数组元素的*比较*

#### 先学习 NumPy 的掩码

### 7.2 通用函数中的比较运算符

前面介绍了通用函数，重点关注的是

- 算术运算，如加、减、乘、除、等

NumPy 还实现了<font color="red">比较运算符</font>，如

- ``<`` (小于) 
- ``>`` (大于)
- 其它

这些都是<font color="red">元素级通用函数</font>，它们产生的结果是<font color="red">布尔类型数组</font>。

In [ ]:
x = np.array([1, 2, 3, 4, 5])

In [ ]:
x < 3  # 小于

In [ ]:
x > 3  # 大于

In [ ]:
x <= 3  # 小于等于

In [ ]:
x >= 3  # 大于等于

In [ ]:
x != 3  # 不等于

In [ ]:
x == 3  # 等于

逐元素比较

In [ ]:
(2 * x) == (x ** 2)

####  比较运算符对应的通用函数

- 与加减乘除相似，在NumPy中，比较运算符也实现为通用函数

例如， 计算 ``x < 3``, 内部将调用函数 ``np.less(x, 3)``

#### 比较运算符及其通用函数列表

| 运算符	    | 通用函数    || 运算符	   | 通用函数    |
|---------------|---------------------||---------------|---------------------|
|``==``         |``np.equal``         ||``!=``         |``np.not_equal``     |
|``<``          |``np.less``          ||``<=``         |``np.less_equal``    |
|``>``          |``np.greater``       ||``>=``         |``np.greater_equal`` |

#### 适用于任意的数组形状

- 正如算术类通用函数，这些运算符也适用于任意尺寸和形状的数组

考虑一二维数组算例

In [ ]:
rng = np.random.RandomState(0)
x = rng.randint(10, size=(3, 4))
x

In [ ]:
x < 6

### 7.3 布尔数组操作

给定一布尔数组，可进行多种操作

In [ ]:
print(x)

#### 计数操作

- 对真值计数，使用``np.count_nonzero``

In [ ]:
# 计数有多少小于 6?
np.count_nonzero(x < 6)

#### 对指定条件求和

- 使用``np.sum``函数

这里，``False``按``0``对待，``True``按``1``对待:

In [ ]:
np.sum(x < 6)

使用``sum()``的好处还有：

- NumPy 聚合函数可沿行进行，也可沿列进行

In [ ]:
# 每行有多少值小于 6 ?
np.sum(x < 6, axis=1)

这里，计算出矩阵各行小于 6 的元素个数

#### ``np.any`` 和 ``np.all``

用途

- 快速检查是否有真值
- 或所有元素是否无真值

In [ ]:
x

In [ ]:
# x 中是否有任何值大于 8?
np.any(x > 8)

In [ ]:
# x 中是否有任何值小于 0?
np.any(x < 0)

In [ ]:
# x 中是否所有值小于 10?
np.all(x < 10)

In [ ]:
# x 中是否有所有值等于 6?
np.all(x == 6)

#### 用于特定轴

- ``np.all``和``np.any``也可用在特定轴上

例如

In [ ]:
# 对于每行，是否所有值都小于 8 ?
print(x)
np.all(x < 8, axis=1)

第1和第3行的所有元素均小于8，而第2行却不是。

注意：

- Python 有若干内建函数，如``sum()``、``any()``和``all()``
- 在NumPy中也有同样功能的函数
- Python 内建函数用于多维数组会产生不可预测的结果
- 确保对多维数组使用``np.sum()``、``np.any()``和``np.all()``

### 7.4 布尔运算符

回到Seattle，关心

- 多少天降雨量小于4英寸且大于1英寸？
- 使用*位逻辑运算符*:``&``、``|``、``^``和``~``

与前相同，NumPy也重载了这些通用函数

In [ ]:
np.sum((inches > 0.5) & (inches < 1))

#### 注意

这里，小括号很重要，若省略，将会是

``` python
inches > (0.5 & inches) < 1
```

还可采用等效的逻辑式 *A AND B* and *NOT (NOT A OR NOT B)* 

#### 发现

- 有29天的降雨量介于 0.5 和 1.0 英寸之间

In [ ]:
np.sum(~( (inches <= 0.5) | (inches >= 1) ))

#### 逻辑表达式

- 结合比较运算符和逻辑运算符，可以有更多样的逻辑表达式

下表给出按位操作的布尔运算符及其等效通用函数

| 运算符	    | 通用函数    || 运算符	    |通用函数    |
|---------------|---------------------||---------------|---------------------|
|``&``          |``np.bitwise_and``   ||&#124;         |``np.bitwise_or``    |
|``^``          |``np.bitwise_xor``   ||``~``          |``np.bitwise_not``   |

采用这些工具，我们可能回答前面关于 Seattle 天气数据的问题。

In [ ]:
print("Number days without rain:      ", np.sum(inches == 0))
print("Number days with rain:         ", np.sum(inches != 0))
print("Days with more than 0.5 inches:", np.sum(inches > 0.5))
print("Rainy days with < 0.2 inches  :", np.sum((inches > 0) &
                                                (inches < 0.2)))

### 7.5 布尔掩码数组

- 前面介绍了利用聚合函数直接运算布尔数组
- 更强大的模式是利用掩码来选取特定的数据子集

In [ ]:
x

按给定条件，得到一布尔数组

In [ ]:
x < 5

现在，从上述布尔数组选取值，这就是所谓 <font color="red">*掩码* 操作</font>

In [ ]:
x[x < 5]

#### 注意

- 返回的是一维数组，每个元素对应的掩码都是 ``True``

然后，即可进行需要的计算

In [ ]:
# construct a mask of all rainy days
rainy = (inches > 0)

# construct a mask of all summer days (June 21st is the 172nd day)
days = np.arange(365)
summer = (days > 172) & (days < 262)

print("Median precip on rainy days in 2014 (inches):   ",
      np.median(inches[rainy]))
print("Median precip on summer days in 2014 (inches):  ",
      np.median(inches[summer]))
print("Maximum precip on summer days in 2014 (inches): ",
      np.max(inches[summer]))
print("Median precip on non-summer rainy days (inches):",
      np.median(inches[rainy & ~summer]))

#### 对 Seattle 天气量化判断的基本方法

- 布尔操作
- 掩码运算
- 聚合函数

快速回答了这类问题

### 7.6 进一步的讨论

- 应该使用关键字 and/or 还是运算符 &/|

#### 容易混淆的问题是

- 关键字and/or 和运算符 &/|有什么区别？
- 何时该使用哪一类？

#### 区别
- ``and`` 和 ``or`` 针对整个对象
- ``&`` 和 ``|`` 针对 *对象的位*

#### 注意

- 使用``and`` 或 ``or``时，要求 Python 把对象<font color="red">当成单一的布尔实体来处理</font>
- 在 Python, 所有的非零值被当成 True

In [ ]:
bool(42), bool(0)

In [ ]:
bool(42 and 0)

In [ ]:
bool(42 or 0)

对整数使用``&`` 和 ``|``时, 是按位来操作

In [ ]:
bin(42)

In [ ]:
bin(59)

In [ ]:
bin(42 & 59)

In [ ]:
bin(42 | 59)

注意：对相应的位进行比较，得到结果

In [ ]:
A = np.array([1, 0, 1, 0, 1, 0], dtype=bool)
B = np.array([1, 1, 1, 0, 1, 1], dtype=bool)
A | B

对数组使用``or``，即判别整个数组对象的真假

In [ ]:
A or B

类似地，对数组进行布尔运算，必须采用 ``|`` 或 ``&``，而不是``or`` 或 ``and``:

In [ ]:
x = np.arange(10)
(x > 4) & (x < 8)

不能如下式用``and``

In [ ]:
(x > 4) and (x < 8)

### 小结

- ``and`` 和 ``or`` 对单个对象进行操作
- ``&`` 和 ``|`` 对多项内容进行布尔运算操作

而对 NumPy 数组，后者几乎总是期望的操作
For Boolean NumPy arrays, the latter is nearly always the desired operation.

### 结束